In [1]:
import pandas as pd


In [2]:
wd = pd.read_csv("../world_data.csv")
wp = pd.read_csv("../world_population.csv")

In [3]:
wp = wp.rename(columns={'Country':'Country/Region'})

In [9]:
_DATE_7 = pd.Timestamp('now').floor('D') + pd.offsets.Day(-7)
_DATE_14 = pd.Timestamp('now').floor('D') + pd.offsets.Day(-14)
_DATE_30 = pd.Timestamp('now').floor('D') + pd.offsets.Day(-30)


def calc_per_million(amount, population):
    if amount is None:
        return 0
    
    if population is None or population == 0:
       return 0
       
    return amount * 1000000 / population



In [5]:
wp = wp.rename(columns={'Country':'Country/Region'})
wd = pd.merge(wd, wp, on=['Country/Region'])


In [6]:

#TODO 
wd['MyCountries'] = False    

#covernt column to datetime
wd['Date'] = pd.to_datetime(wd['Date'])
wd['WeekNum'] = wd.Date.dt.week    
wd["Month"] = wd.Date.dt.month
# wd['DayOfYear'] = wd.Date.dt.dayofyear



In [7]:
#set booleans
wd["Last7Days"]  = (wd.Date >= _DATE_7) 
wd["Last14Days"] = (wd.Date >= _DATE_14) 
wd["Last30Days"] = (wd.Date >= _DATE_30) 

In [10]:
wd['CasesPM'] = wd.apply(lambda row : calc_per_million(row['Confirmed'], row['Population']),axis=1)

In [12]:
wd['DeathsPM'] = wd.apply(lambda row: calc_per_million(row['Deaths'], row['Population']),axis=1)

In [13]:
wd.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,MyCountries,WeekNum,Month,Last7Days,Last14Days,Last30Days,CasesPM,DeathsPM
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.0,0.0
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.0,0.0
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.0,0.0
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.0,0.0
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.0,0.0


In [14]:

#calculate rate of chage
# g = wd.groupby(['Country/Region','Date'])?
g = wd.sort_values('Date').groupby(['Country/Region','Province/State'])
wd["GrowthPerDay"] = g.Confirmed.diff()
wd["PercentChange"] = g.Confirmed.pct_change()
wd["DeathPerDay"] = g.Deaths.diff()
wd["DeathPerChange" ] = g.Deaths.pct_change()


In [15]:
g.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,MyCountries,WeekNum,Month,Last7Days,Last14Days,Last30Days,CasesPM,DeathsPM
0,2020-01-22,Afghanistan,NaN,33.000000,65.000000,0.0,0.0,0.0,38928.341,False,4,1,False,False,False,0.000000,0.0
19656,2020-01-22,Peru,NaN,-9.190000,-75.015200,0.0,0.0,0.0,32971.846,False,4,1,False,False,False,0.000000,0.0
15080,2020-01-22,Jordan,NaN,31.240000,36.510000,0.0,0.0,0.0,10203.140,False,4,1,False,False,False,0.000000,0.0
25168,2020-01-22,Yemen,NaN,15.552727,48.516388,0.0,0.0,0.0,29825.968,False,4,1,False,False,False,0.000000,0.0
6136,2020-01-22,China,Guangdong,23.341700,113.424400,26.0,0.0,0.0,1439323.774,False,4,1,False,False,False,18.064038,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,2020-01-26,China,Hunan,27.610400,111.708800,69.0,0.0,0.0,1439323.774,False,4,1,False,False,False,47.939179,0.0
24132,2020-01-26,United Kingdom,Cayman Islands,19.313300,-81.254600,0.0,0.0,0.0,67886.004,False,4,1,False,False,False,0.000000,0.0
8844,2020-01-26,China,Yunnan,24.974000,101.487000,16.0,0.0,0.0,1439323.774,False,4,1,False,False,False,11.116331,0.0
12276,2020-01-26,France,Saint Barthelemy,17.900000,-62.833300,0.0,0.0,0.0,65273.512,False,4,1,False,False,False,0.000000,0.0


In [16]:
#select one country and state for debug
sel =  wd[(wd['Country/Region']=='France') & (wd['Province/State']=='St Martin') ]

In [17]:
count = 0
for index,row in sel.iterrows() :  
    date = row['Date']
    c = row['Confirmed']
    
    cc = row['GrowthPerDay']
    pc = row['PercentChange']
    d = row['Deaths']
    cd = row['DeathPerDay']
    pd = row['DeathPerChange']
    
    print(wd['CasesPM'],)
    
    print(f"{date} c={(c,cc,pc)} d={(d,cd,pd)}")
    count += 1

2020-01-22 00:00:00 c=(0.0, nan, nan) d=(0.0, nan, nan)
2020-01-23 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-24 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-25 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-26 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-27 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-28 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-29 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-30 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-01-31 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-01 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-02 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-03 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-04 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-05 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-06 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-07 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.0, nan)
2020-02-08 00:00:00 c=(0.0, 0.0, nan) d=(0.0, 0.

In [ ]:
print(count)